In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [4]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [5]:
def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = tweet.replace("#","")
    tweet = re.sub(r'http\S+', '',tweet)
    tweet = "".join(i for i in tweet if ord(i)<128)
    tweet = tweet.replace("[^a-zA-Z0-9]", " ")
    tweet = [wordninja.split(word) for word in tweet.split()]
    tweet = " ".join([item for items in tweet for item in items])
    return tweet

In [8]:
import pandas as pd 
import re
import wordninja
train_df = pd.read_csv("train_2kmZucJ.csv")
test_df  = pd.read_csv("test_oJQbWVk.csv")

test_df['clean_tweet'] = test_df['tweet'].apply(clean_tweet,1)
train_df['clean_tweet'] = train_df['tweet'].apply(clean_tweet,1)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
word_seq_train = tokenizer.texts_to_sequences(X_train)
word_seq_test = tokenizer.texts_to_sequences(X_test)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df.clean_tweet, train_df.label, stratify = train_df.label , test_size = 0.1)

In [18]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

In [25]:
# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [26]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 500  # Only consider the first 200 words of each movie review

In [27]:
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [28]:
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(x)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=text_input, outputs=outputs)

In [29]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])


In [31]:
history = model.fit(
    X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test)
)

Epoch 1/20
223/223 [==============================] - 98s 438ms/step - loss: 0.5742 - accuracy: 0.7442 - val_loss: 0.5712 - val_accuracy: 0.7437
Epoch 2/20
223/223 [==============================] - 90s 401ms/step - loss: 0.5740 - accuracy: 0.7442 - val_loss: 0.5731 - val_accuracy: 0.7437
Epoch 3/20
223/223 [==============================] - 92s 414ms/step - loss: 0.5738 - accuracy: 0.7442 - val_loss: 0.5695 - val_accuracy: 0.7437
Epoch 4/20
223/223 [==============================] - 86s 384ms/step - loss: 0.5725 - accuracy: 0.7444 - val_loss: 0.5692 - val_accuracy: 0.7437
Epoch 5/20
223/223 [==============================] - 90s 404ms/step - loss: 0.5735 - accuracy: 0.7442 - val_loss: 0.5702 - val_accuracy: 0.7437
Epoch 6/20
223/223 [==============================] - 84s 375ms/step - loss: 0.5715 - accuracy: 0.7442 - val_loss: 0.5698 - val_accuracy: 0.7437
Epoch 7/20
223/223 [==============================] - 84s 376ms/step - loss: 0.5712 - accuracy: 0.7442 - val_loss: 0.5699 - val_ac

KeyboardInterrupt: 